# Libraries

In [1]:
import os, glob
import random
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd
import multiprocessing
from copy import deepcopy
from sklearn.metrics import precision_recall_curve, auc
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from keras.utils import Sequence
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
import matplotlib.pyplot as plt
from IPython.display import Image
from tqdm import tqdm_notebook as tqdm
from numpy.random import seed
seed(10)
from tensorflow import set_random_seed
set_random_seed(10)
%matplotlib inline

Using TensorFlow backend.


In [20]:
test_imgs_folder = './input/test_images/'
train_imgs_folder = './input/train_images/'
num_cores = multiprocessing.cpu_count()
print(num_cores)

8


# Data Generators

## One-hot encoding classes

In [3]:
train_df = pd.read_csv('./input/train.csv')
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [4]:
train_df = train_df[~train_df['EncodedPixels'].isnull()]
train_df['Image'] = train_df['Image_Label'].map(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].map(lambda x: x.split('_')[1])
classes = train_df['Class'].unique()
train_df = train_df.groupby('Image')['Class'].agg(set).reset_index()
for class_name in classes:
    train_df[class_name] = train_df['Class'].map(lambda x: 1 if class_name in x else 0)
train_df.head()

,Image,Class,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Flower, Fish}",1,1,0,0
1,002be4f.jpg,"{Sugar, Flower, Fish}",1,1,1,0
2,0031ae9.jpg,"{Sugar, Flower, Fish}",1,1,1,0
3,0035239.jpg,"{Gravel, Flower}",0,1,0,1
4,003994e.jpg,"{Gravel, Sugar, Fish}",1,0,1,1


In [5]:
# dictionary for fast access to ohe vectors
img_2_ohe_vector = {img:vec for img, vec in zip(train_df['Image'], train_df.iloc[:, 2:].values)}

In [38]:
img_2_ohe_vector

{'0011165.jpg': array([1, 1, 0, 0], dtype=int64),
 '002be4f.jpg': array([1, 1, 1, 0], dtype=int64),
 '0031ae9.jpg': array([1, 1, 1, 0], dtype=int64),
 '0035239.jpg': array([0, 1, 0, 1], dtype=int64),
 '003994e.jpg': array([1, 0, 1, 1], dtype=int64),
 '00498ec.jpg': array([0, 0, 0, 1], dtype=int64),
 '006bf7c.jpg': array([1, 0, 1, 0], dtype=int64),
 '006c5a6.jpg': array([1, 0, 1, 0], dtype=int64),
 '008233e.jpg': array([0, 0, 1, 0], dtype=int64),
 '008a5ff.jpg': array([1, 0, 1, 0], dtype=int64),
 '0091591.jpg': array([0, 1, 1, 1], dtype=int64),
 '0095357.jpg': array([0, 0, 1, 0], dtype=int64),
 '009e2f3.jpg': array([1, 0, 1, 1], dtype=int64),
 '00a0954.jpg': array([0, 0, 1, 1], dtype=int64),
 '00b81e1.jpg': array([0, 1, 1, 1], dtype=int64),
 '00bea06.jpg': array([1, 0, 0, 1], dtype=int64),
 '00cedfa.jpg': array([0, 0, 1, 1], dtype=int64),
 '00d4443.jpg': array([0, 0, 1, 0], dtype=int64),
 '00dec6a.jpg': array([1, 1, 1, 1], dtype=int64),
 '0100a84.jpg': array([1, 0, 0, 1], dtype=int64),


## Stratified split into train/val

In [6]:
train_imgs, val_imgs = train_test_split(train_df['Image'].values, 
                                        test_size=0.2, 
                                        stratify=train_df['Class'].map(lambda x: str(sorted(list(x)))), # sorting present classes in lexicographical order, just to be sure
                                        random_state=17)

## Generator class

In [23]:
class DataGenenerator(Sequence):
    def __init__(self, images_list=None, folder_imgs=train_imgs_folder, 
                 batch_size=16, shuffle=True, augmentation=None,
                 resized_height=352, resized_width=544, num_channels=3):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        if images_list is None:
            self.images_list = os.listdir(folder_imgs)
        else:
            self.images_list = deepcopy(images_list)
        self.folder_imgs = folder_imgs
        self.len = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.is_test = not 'train' in folder_imgs
        if not shuffle and not self.is_test:
            self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.len*self.batch_size]]

    def __len__(self):
        return self.len
    
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)

    def __getitem__(self, idx):
        current_batch = self.images_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))
        y = np.empty((self.batch_size, self.num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(self.folder_imgs, image_name)
            img = cv2.resize(cv2.imread(path), (self.resized_width, self.resized_height), interpolation=cv2.INTER_AREA).astype(np.float32)
            if not self.augmentation is None:
                augmented = self.augmentation(image=img)
                img = augmented['image']
            X[i, :, :, :] = img/255.0
            if not self.is_test:
                y[i, :] = img_2_ohe_vector[image_name]
        return X, y

    def get_labels(self):
        if self.shuffle:
            images_current = self.images_list[:self.len*self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

In [24]:
albumentations_train = Compose([
    VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion()
], p=1)

Generator instances

In [33]:
data_generator_train = DataGenenerator(train_imgs, augmentation=albumentations_train, batch_size=4)
data_generator_train_eval = DataGenenerator(train_imgs, shuffle=False, batch_size=4)
data_generator_val = DataGenenerator(val_imgs, shuffle=False, batch_size=4)

# Loss functions and variable learning rates

The callback would be used:
1. to estimate AUC under precision recall curve for each class,
2. to early stop after 5 epochs of no improvement in mean PR AUC,
3. save a model with the best PR AUC in validation,
4. to reduce learning rate on PR AUC plateau.

In [26]:
import keras.backend as K
from keras.losses import binary_crossentropy, mean_squared_error

def dice_coeff(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    loss = 0.5 * binary_crossentropy(y_true, y_pred) - 2*dice_coeff(y_true, y_pred)
    return loss

def plateau_then_triangular_lr(epocs):
    plateau_steps = 4
    step_size = 2 
    min_lr = 1e-5
    max_lr = 1e-2
    
    if epocs < plateau_steps:
        return (min_lr + max_lr)/2
    
    cycle = np.floor(1+epocs/(2*step_size))
    x = np.abs(epocs/step_size - 2*cycle + 1)
    lr = min_lr + (max_lr - min_lr) * np.maximum(0, (1-x))/float(1.0**(cycle-1))
    print("NEW learing rate: " + str(lr))
    return lr

def plateau_then_decl_triangular_lr(epocs):
    plateau_steps = 3
    step_size = 2
    min_lr = 1e-6 #1e-5
    max_lr = 5e-4 #5e-3
    
    if epocs < plateau_steps:
        return (min_lr + max_lr)/2
    
    cycle = np.floor(1+epocs/(2*step_size))
    x = np.abs(epocs/step_size - 2*cycle + 1)
    lr = min_lr + (max_lr - min_lr) * np.maximum(0, (1-x))/float(1.2**(cycle-1))
    print("NEW learing rate: " + str(lr))
    return lr


# Classifier

## Defining a model

In [27]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization

def unet_352x544_segmentation_model(input_shape=(352, 544, 3), num_classes=1):
    inputs = Input(shape=input_shape)

    down_1 = Conv2D(16, (3, 3), padding='same')(inputs)
    down_1 = BatchNormalization()(down_1)
    down_1 = Activation('relu')(down_1)
    down_1 = Conv2D(16, (3, 3), padding='same')(down_1)
    down_1 = BatchNormalization()(down_1)
    down_1 = Activation('relu')(down_1)
    down_1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down_1) # output 176, 272
    
    down0 = Conv2D(32, (3, 3), padding='same')(down_1_pool)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0) # 88, 136

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1) # 44, 68

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2) # 22, 34

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3) # 11, 17

    center = Conv2D(1024, (3, 3), padding='same')(down3_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center) # center

    up3 = UpSampling2D((2, 2))(center)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2) 

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)

    up_1 = UpSampling2D((2, 2))(up0)
    up_1 = concatenate([down_1, up_1], axis=3)
    up_1 = Conv2D(16, (3, 3), padding='same')(up_1)
    up_1 = BatchNormalization()(up_1)
    up_1 = Activation('relu')(up_1)
    up_1 = Conv2D(16, (3, 3), padding='same')(up_1)
    up_1 = BatchNormalization()(up_1)
    up_1 = Activation('relu')(up_1)
    up_1 = Conv2D(16, (3, 3), padding='same')(up_1)
    up_1 = BatchNormalization()(up_1)
    up_1 = Activation('relu')(up_1)

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up_1)

    model = Model(inputs=inputs, outputs=classify)
    
    return model

In [28]:
from keras.optimizers import RMSprop

model = unet_352x544_segmentation_model(num_classes=4)
model.compile(optimizer=RMSprop(lr=0.0003), loss=bce_dice_loss, metrics=[dice_coeff, 'accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 352, 544, 3)  0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 352, 544, 16) 448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 352, 544, 16) 64          conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 352, 544, 16) 0           batch_normalization_55[0][0]     
__________________________________________________________________________________________________
conv2d_58 

__________________________________________________________________________________________________
batch_normalization_66 (BatchNo (None, 11, 17, 1024) 4096        conv2d_68[0][0]                  
__________________________________________________________________________________________________
activation_66 (Activation)      (None, 11, 17, 1024) 0           batch_normalization_66[0][0]     
__________________________________________________________________________________________________
up_sampling2d_11 (UpSampling2D) (None, 22, 34, 1024) 0           activation_66[0][0]              
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 22, 34, 1280) 0           activation_64[0][0]              
                                                                 up_sampling2d_11[0][0]           
__________________________________________________________________________________________________
conv2d_69 

__________________________________________________________________________________________________
activation_76 (Activation)      (None, 176, 272, 32) 0           batch_normalization_76[0][0]     
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 176, 272, 32) 9248        activation_76[0][0]              
__________________________________________________________________________________________________
batch_normalization_77 (BatchNo (None, 176, 272, 32) 128         conv2d_79[0][0]                  
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 176, 272, 32) 0           batch_normalization_77[0][0]     
__________________________________________________________________________________________________
conv2d_80 (Conv2D)              (None, 176, 272, 32) 9248        activation_77[0][0]              
__________

## Initial tuning of the added fully-connected layer

After unfreezing all the layers(except last 3) I set a less aggressive initial learning rate and train until early stopping (or 100 epochs max).

In [29]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    dice_coeff = history.history['dice_coeff']
    val_dice_coeff = history.history['val_dice_coeff']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')

    plt.figure()

    plt.plot(epochs, dice_coeff, 'r.')
    plt.plot(epochs, val_dice_coeff, 'r-')
    plt.title('Training and validation Dice coef.')

    plt.show()

In [36]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard

model_file_name = 'Unet_352x544_differnet_optimizers_etc_v1'

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=2, min_delta=1e-3),
             LearningRateScheduler(plateau_then_decl_triangular_lr),
             ModelCheckpoint(monitor='val_loss', filepath='./checkpoints/' + str(model_file_name) + '.h5', 
                             save_best_only=True, save_weights_only=False)]

history_1 = model.fit_generator(generator=data_generator_train,
                              validation_data=data_generator_val,
                              epochs=10,
                              callbacks=callbacks,
                              workers=num_cores,
                              verbose=1
                             )

Epoch 1/10


InvalidArgumentError: Incompatible shapes: [16] vs. [3063808]
	 [[{{node loss_1/conv2d_84_loss/mul_1}}]]
	 [[{{node metrics_1/acc/Mean}}]]

In [ ]:
plot_training(history_1)

In [ ]:
# EXTRA EPOCHS
history_1_1 = model.fit_generator(generator=data_generator_train,
                              validation_data=data_generator_val,
                              epochs=8,
                              callbacks=callbacks,
                              workers=num_cores,
                              verbose=1
                             )

In [ ]:
plot_training(history_1_1)

In [ ]:
from keras.optimizers import SGD

model_file_name = 'Efficientnetb3_350x525_SGD_v1'

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=2, min_delta=1e-3),
             ModelCheckpoint(monitor='val_loss', filepath='./checkpoints/' + str(model_file_name) + '.h5', 
                             save_best_only=True, save_weights_only=False)]

sgd = SGD(lr=1e-4, decay=3e-6, momentum=0.92, nesterov=True)
model.compile(optimizer=sgd, loss=bce_dice_loss, metrics=['accuracy', dice_coeff])
history_1_2 = model.fit_generator(generator=data_generator_train,
                              validation_data=data_generator_val,
                              epochs=5,
                              callbacks=callbacks,
                              workers=num_cores,
                              verbose=1
                             )

In [ ]:
plot_training(history_1_2)

# Selecting postprocessing thresholds

In [ ]:
class_names = ['Fish', 'Flower', 'Sugar', 'Gravel']
def get_threshold_for_recall(y_true, y_pred, class_i, recall_threshold=0.94, precision_threshold=0.95, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true[:, class_i], y_pred[:, class_i])
    i = len(thresholds) - 1
    best_recall_threshold = None
    while best_recall_threshold is None:
        next_threshold = thresholds[i]
        next_recall = recall[i]
        if next_recall >= recall_threshold:
            best_recall_threshold = next_threshold
        i -= 1
        
    # consice, even though unnecessary passing through all the values
    best_precision_threshold = [thres for prec, thres in zip(precision, thresholds) if prec >= precision_threshold]
    if not best_precision_threshold:
        best_precision_threshold = 0.95
    else:
        best_precision_threshold = best_precision_threshold[0]
    
    if plot:
        plt.figure(figsize=(10, 7))
        plt.step(recall, precision, color='r', alpha=0.3, where='post')
        plt.fill_between(recall, precision, alpha=0.3, color='r')
        plt.axhline(y=precision[i + 1])
        recall_for_prec_thres = [rec for rec, thres in zip(recall, thresholds) if thres == best_precision_threshold]
        if not recall_for_prec_thres:
            recall_for_prec_thres = 0.94
        else:
            recall_for_prec_thres = recall_for_prec_thres[0]
                                                            
        plt.axvline(x=recall_for_prec_thres, color='g')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.legend(['PR curve', 
                    f'Precision {precision[i + 1]: .2f} corresponding to selected recall threshold',
                    f'Recall {recall_for_prec_thres: .2f} corresponding to selected precision threshold'])
        plt.title(f'Precision-Recall curve for Class {class_names[class_i]}')
    return best_recall_threshold, best_precision_threshold

y_pred = model.predict_generator(data_generator_val, workers=num_cores)
y_true = data_generator_val.get_labels()
recall_thresholds = dict()
precision_thresholds = dict()
for i, class_name in tqdm(enumerate(class_names)):
    recall_thresholds[class_name], precision_thresholds[class_name] = get_threshold_for_recall(y_true, y_pred, i, plot=True)

# Post-processing segmentation submission

Predicting cloud classes for test.

In [ ]:
precision_thresholds

In [ ]:
recall_thresholds

{'Fish': 0.01, 'Flower': 0.08, 'Sugar': 0.05, 'Gravel': 0.07} gave 0.6610

In [ ]:
recall_thresholds['Gravel'] = 0.07
recall_thresholds['Sugar'] = 0.05
recall_thresholds['Flower'] = 0.08
recall_thresholds['Fish'] = 0.01


In [ ]:
data_generator_test = DataGenenerator(folder_imgs=test_imgs_folder, shuffle=False)
y_pred_test = model.predict_generator(data_generator_test, workers=num_cores)

Estimating set of images without masks.

In [ ]:
image_labels_empty = set()
for i, (img, predictions) in enumerate(zip(os.listdir(test_imgs_folder), y_pred_test)):
    for class_i, class_name in enumerate(class_names):
        if predictions[class_i] < recall_thresholds[class_name]:
            image_labels_empty.add(f'{img}_{class_name}')

Segmentation results:

In [ ]:
submission = pd.read_csv('./input/cloudefficientnetb2/submission.csv')
submission.head()

In [ ]:
predictions_nonempty = set(submission.loc[~submission['EncodedPixels'].isnull(), 'Image_Label'].values)

In [ ]:
print(f'{len(image_labels_empty.intersection(predictions_nonempty))} masks would be removed')

In [ ]:
#removing masks
submission.loc[submission['Image_Label'].isin(image_labels_empty), 'EncodedPixels'] = np.nan
submission.to_csv('submission_350x525_differnet_optimizers.csv', index=None)

# Future work
1. estimate distribution of classes in test set using the classifier. Then, if necessary and doable, modify val set accordingly,
2. use the classifier with explainability technique [Gradient-weighted Class Activation Mapping](http://gradcam.cloudcv.org/) to generate a baseline, (please see [GradCAM: extracting masks from classifier](https://www.kaggle.com/samusram/gradcam-extracting-masks-from-classifier)),
3. improve the classifier,
4. use the classifier as backbone for UNet-like solution.